In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
import graphviz
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

## Data Treatment

In [3]:
df = pd.read_csv("/kaggle/input/titanic/train.csv")

In [4]:
def split_dataset(dataset, test_ratio=0.28):
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

dft, dfv = split_dataset(df)
print("{} examples in training, {} examples in testing.".format(
    len(dft), len(dfv)))

640 examples in training, 251 examples in testing.


In [5]:
y= dft['Survived']
Y=dfv['Survived']
x = dfv.drop(['Survived','Name','Ticket','Cabin'],axis=1)
X=dft.drop(['Survived','Name','Ticket','Cabin'],axis=1)

In [6]:
X['Embarked'] = X['Embarked'].fillna('S')
X['Age'] = X['Age'].fillna(round(df.Age.mean()))
X['Sex'] = X['Sex'].replace({'male': 0, 'female': 1})
X['Embarked'] = X['Embarked'].replace({'S': 0, 'C': 1,'Q': 2})

/tmp/ipykernel_18/2973458064.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X['Sex'] = X['Sex'].replace({'male': 0, 'female': 1})
/tmp/ipykernel_18/2973458064.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X['Embarked'] = X['Embarked'].replace({'S': 0, 'C': 1,'Q': 2})


In [7]:
x['Embarked'] = x['Embarked'].fillna('S')
x['Age'] = x['Age'].fillna(round(df.Age.mean()))
x['Sex'] = x['Sex'].replace({'male': 0, 'female': 1})
x['Embarked'] = x['Embarked'].replace({'S': 0, 'C': 1,'Q': 2})

/tmp/ipykernel_18/1346004681.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x['Sex'] = x['Sex'].replace({'male': 0, 'female': 1})
/tmp/ipykernel_18/1346004681.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x['Embarked'] = x['Embarked'].replace({'S': 0, 'C': 1,'Q': 2})


## Modeling

### Decision Tree Classifier

In [8]:
model=DecisionTreeClassifier(max_depth=5)

In [9]:
model.fit(X,y)

DecisionTreeClassifier(max_depth=5)

In [10]:
trueval = dfv['Survived']
predval = model.predict(x)
accuracy_score(trueval, predval)

0.7928286852589641

### XG Boost Random Forest Classifier

### First Hyperparameter tuning

In [11]:
import optuna
import sklearn.metrics


dtrain = xgb.DMatrix(X, label=dft['Survived'])
dvalid = xgb.DMatrix(x, label=dfv['Survived'])

def objective(trial):
    optuna.logging.set_verbosity(optuna.logging.DEBUG)
    param = {
        'objective': 'binary:logistic',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth',1, 100),
        'max_leaves': trial.suggest_int('max_depth',1, 100),
        'n_estimators': trial.suggest_int('n_estimators',1, 100),
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        'learning_rate': trial.suggest_float('learning_rate', 1e-8, 1.0, log=True)
    }    
    bst = xgb.train(param, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(dfv['Survived'], pred_labels)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2024-03-06 23:39:19,578] A new study created in memory with name: no-name-80d8c070-8030-4ef0-989b-97a187f1e5c8
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [23:39:19] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-03-06 23:39:19,629] Trial 0 finished with value: 0.601593625498008 and parameters: {'booster': 'dart', 'gamma': 1.569621505929269e-07, 'max_depth': 11, 'n_estimators': 43, 'grow_policy': 'lossguide', 'learning_rate': 2.036771780336469e-08}. Best is trial 0 with value: 0.601593625498008.
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [23:39:19] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "gamma", "grow_policy", "max_depth", "max_leaves", "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-03-06 23:39:19,642] Trial 1 finished with value: 0.601593625498008 and parameters: {'booster': 'gblinear', 'gamma': 1

In [12]:
m2 = xgb.XGBRFClassifier(
    booster='gbtree',
    device='cpu',
    eval_metric=mean_absolute_error,
    grow_policy = 'lossguide',
    gamma = 2.074237418451085e-08,
    max_depth= 52,
    n_estimators = 56,
    learning_rate = 0.3010386961800788
)

In [13]:
m2.fit(X,y,eval_set=[(X,y),(x,Y)])

[0]	validation_0-logloss:0.50099	validation_0-mean_absolute_error:0.38328	validation_1-logloss:0.55090	validation_1-mean_absolute_error:0.40782


XGBRFClassifier(base_score=None, booster='gbtree', callbacks=None,
                colsample_bylevel=None, colsample_bytree=None, device='cpu',
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=<function mean_absolute_error at 0x79c1fff28b80>,
                feature_types=None, gamma=2.074237418451085e-08,
                grow_policy='lossguide', importance_type=None,
                interaction_constraints=None, learning_rate=0.3010386961800788,
                max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_depth=52, max_leaves=None,
                min_child_weight=None, missing=nan, monotone_constraints=None,
                multi_strategy=None, n_estimators=56, n_jobs=None,
                num_parallel_tree=None, objective='binary:logistic',
                random_state=None, ...)

In [14]:
trueval = dfv['Survived']
predval = m2.predict(x)
accuracy_score(trueval, predval)

0.796812749003984

## Prediction

### Test DF predictions

In [15]:
dftest = pd.read_csv('/kaggle/input/titanic/test.csv')
xtest=dftest.drop(['Name','Ticket','Cabin'],axis=1)

In [16]:
xtest['Embarked'] = xtest['Embarked'].fillna('S')
xtest['Age'] = xtest['Age'].fillna(round(xtest.Age.mean()))
xtest['Sex'] = xtest['Sex'].replace({'male': 0, 'female': 1})
xtest['Embarked'] = xtest['Embarked'].replace({'S': 0, 'C': 1,'Q': 2})
xtest['Fare'] = xtest['Fare'].fillna(xtest.Fare.median())

/tmp/ipykernel_18/392077192.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  xtest['Sex'] = xtest['Sex'].replace({'male': 0, 'female': 1})
/tmp/ipykernel_18/392077192.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  xtest['Embarked'] = xtest['Embarked'].replace({'S': 0, 'C': 1,'Q': 2})


In [17]:
dftest['Survived'] = m2.predict(xtest)

In [18]:
dfexport = dftest[['PassengerId','Survived']]
dfexport.to_csv('predictionoutput3.csv')